In [1]:
import os
os.chdir('/Users/niteshkumarsharma/Desktop/Folder/AI/NanoGPTImplemenation')

## Reading ShakesPeare Data

In [2]:
with open("data/input.txt", "r", encoding='utf-8') as f:
    text = f.read()

In [3]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


## Tokenisation into Characters

In [4]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

print(" ".join(chars))
print(vocab_size)


   ! $ & ' , - . 3 : ; ? A B C D E F G H I J K L M N O P Q R S T U V W X Y Z a b c d e f g h i j k l m n o p q r s t u v w x y z
65


### Encoding 

In [5]:
## string to indexes
stoi = {ch:i for i, ch in enumerate(chars)}

## indexes to string 
itos = {i : ch for i, ch in enumerate(chars)}


In [6]:
encode = lambda s: [stoi[c] for c in s] ## encoder: takes a string, output list of indexes/integrs
decode = lambda l: ''.join([itos[i] for i in l]) ## decoder: take a list of integers, output a string

In [7]:
print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [8]:
decode([46, 47, 47, 1, 58, 46, 43, 56, 43])

'hii there'

In [9]:
len(encode(text))

1115394

### Using PyTorch

In [10]:
import torch

In [11]:
## encoding entire text dataset and storing it into a torch.tensor
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [12]:
## splitting into train and validation set
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

### Defining Context Length/Block Size

**Context Length**: Here context lenth is maximum number of characters we are feeding during backpropagation. Starting from context size of 1 to context size of block size

This allows transformer to see from block size 1 to block size of max context length

Each Block size will contain multiple examples for training as described below

In [13]:
block_size = 8
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [14]:
train_data[:9]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [16]:
x = train_data[:block_size]
y = train_data[1 : block_size + 1]

for t in range(block_size):
    context = x[:t + 1]
    target = y[t]
    print(f"when input is {context} the target: {target}")  

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


### Creating Batches to Keep GPU Busy

In [27]:
torch.manual_seed(1337)

batch_size = 4 # how many independent sequences to process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    data = train_data if split == 'train' else val_data

    ix = torch.randint(len(data) - block_size, (batch_size,))

    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1: i+block_size+1] for i in ix])

    return x,y


In [28]:
xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [41]:
for batch in range(batch_size):
    batch_input = xb[batch]
    batch_output = yb[batch]
    for context in range(block_size):
        x = batch_input[:context + 1]
        y = batch_output[context]
        print(f"when input is {x} the target: {y}")  
        


when input is tensor([24]) the target: 43
when input is tensor([24, 43]) the target: 58
when input is tensor([24, 43, 58]) the target: 5
when input is tensor([24, 43, 58,  5]) the target: 57
when input is tensor([24, 43, 58,  5, 57]) the target: 1
when input is tensor([24, 43, 58,  5, 57,  1]) the target: 46
when input is tensor([24, 43, 58,  5, 57,  1, 46]) the target: 43
when input is tensor([24, 43, 58,  5, 57,  1, 46, 43]) the target: 39
when input is tensor([44]) the target: 53
when input is tensor([44, 53]) the target: 56
when input is tensor([44, 53, 56]) the target: 1
when input is tensor([44, 53, 56,  1]) the target: 58
when input is tensor([44, 53, 56,  1, 58]) the target: 46
when input is tensor([44, 53, 56,  1, 58, 46]) the target: 39
when input is tensor([44, 53, 56,  1, 58, 46, 39]) the target: 58
when input is tensor([44, 53, 56,  1, 58, 46, 39, 58]) the target: 1
when input is tensor([52]) the target: 58
when input is tensor([52, 58]) the target: 1
when input is tensor(

In [30]:
block_size

8

In [ ]:
xb[0,]

tensor([24, 43, 58,  5, 57,  1, 46, 43])

In [35]:
yb[0]

tensor([43, 58,  5, 57,  1, 46, 43, 39])

In [37]:
xb[0,:1+1]

tensor([24, 43])

In [38]:
yb[0, 1]

tensor(58)